In [ ]:
# FewShotPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector
      

# setup streaming Chat model
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
    )

examples = [
    {
    "question": "What're the details about Juror #2?",
    "answer": """
    Here's what I know :)
    "Director: Clint Eastwood"
    "Main Cast: Nicholas Hoult, Toni Collette, J. K. Simmons"
    "Budget: ~$35 million"
    "Genre: Legal thriller"
    "Synopsis: A juror realizes he may have caused the victim’s death."
    """
    },
    {
    "question": "What are the details about Inception?",
    "answer": """
    Here's what I know :)
    "Director: Christopher Nolan"
    "Main Cast: Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page"
    "Budget: ~$160 million"
    "Genre: Science fiction thriller"
    "Synopsis: A thief enters dreams to plant an idea."
    """
    },
    {
    "question": "Can you tell me the key info for Parasite?",
    "answer": """
    Here's what I know :)
    "Director: Bong Joon-ho"
    "Main Cast: Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong"
    "Budget: ~$11 million"
    "Genre: Dark comedy thriller"
    "Synopsis: A poor family schemes to infiltrate a wealthy household."
    """
    },
    {
    "question": "I'd like an overview of The Grand Budapest Hotel—director, cast, budget, genre, and a short synopsis.",
    "answer": """
    Here's what I know :)
    "Director: Wes Anderson"
    "Main Cast: Ralph Fiennes, Tony Revolori, Saoirse Ronan"
    "Budget: ~$25 million"
    "Genre: Comedy drama"
    "Synopsis: A concierge and lobby boy embark on a caper over a stolen painting."
    """
    },
    {
    "question": "What are the details about La La Land?",
    "answer": """
    Here's what I know :)
    "Director: Damien Chazelle"
    "Main Cast: Ryan Gosling, Emma Stone, John Legend"
    "Budget: ~$30 million"
    "Genre: Musical romance"
    "Synopsis: An actress and a jazz musician fall in love while chasing dreams."
    """
    },
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples
        
    def add_example(self, example):
        self.examples.append(example)
        
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]
    
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {movie}?",
    input_variables=["movie"],
)

chain = prompt | chat

chain.invoke({
    "movie": "Amelie"
})

AI: 
    Here's what I know :)
    "Director: Jean-Pierre Jeunet"
    "Main Cast: Audrey Tautou, Mathieu Kassovitz"
    "Genre: Romantic comedy"
    "Synopsis: A quirky and imaginative young woman named Amelie decides to change the lives of those around her for the better while struggling with her own isolation and loneliness."

AIMessage(content='AI: \n    Here\'s what I know :)\n    "Director: Jean-Pierre Jeunet"\n    "Main Cast: Audrey Tautou, Mathieu Kassovitz"\n    "Genre: Romantic comedy"\n    "Synopsis: A quirky and imaginative young woman named Amelie decides to change the lives of those around her for the better while struggling with her own isolation and loneliness."', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125', 'service_tier': 'default'}, id='run--d59ce17d-3d4c-4621-be94-61e99c98713a-0')